# Splunk App for Data Science and Deep Learning - Dask Distributed KMeans

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with Splunk App for Data Science and Deep Learning.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the model development guide in the documentation.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
import dask_ml.cluster

# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
import dask_ml
print("dask_ml version: " + dask_ml.__version__)

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| inputlookup iris.csv <br/>
| fit MLTKContainer mode=stage algo=dask_kmeans k=3 petal_length petal_width sepal_length sepal_width into app:iris_dask_kmeans

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("iris_dask_kmeans")
print(df)
print(param)

## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    
    #client = Client("tcp://127.0.0.1:")
    client = Client(processes=False)
    
    model['dask_client'] = client
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    ddf = dd.from_pandas(df, npartitions=4)  
    # features dataframe
    features = ddf[param['feature_variables']]
    #features.persist()
    k = int(param['options']['params']['k'])
    model['dask_kmeans'] = dask_ml.cluster.KMeans(n_clusters=k, init_max_iter=2, oversampling_factor=10)
    model['dask_kmeans'].fit(features)
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
%time print(fit(model,df,param))

## Stage 4 - apply the model

In [ ]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    #ddf = dd.from_pandas(df[param['feature_variables']], npartitions=4)
    prediction = model['dask_kmeans'].labels_
    #y_hat = prediction.to_dask_dataframe()
    result = pd.DataFrame(prediction.compute())
    model['dask_client'].close()
    return result

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
%time print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    # TODO
    # model['dask_kmeans'].save_model(MODEL_DIRECTORY + name + '.json')
    return model

In [ ]:
save(model,'dask_kmeans')

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    # TODO
    model = {}
    return model

In [ ]:
loaded_model = load('dask_kmeans')
loaded_model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"model": model}
    return returns

In [ ]:
summary(model)

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code